In [1]:
import MDAnalysis as md_analysis
import matplotlib.pyplot as pyplot
import numpy
from matplotlib.legend import Legend
from numpy import ndarray

## MDAnalysis Universe Class
* The MDAnalysis Universe class contains all the information describing the system.
* There are no required constructor arguments, but to generating a universe will generally require a topology file containing atomic information (in this case `./cnt_atomic.data`), and optionally an MD simulation trajactory/coordinate file (in this case `./cnt_breakable_bonds_dump.lammpstrj`)
* However, some topology files can contain both atomic information **and** coordinate data (e.g. XYZ, PDB, GRO or CRD files)
* Additionally, any file you provide you need to add an additional constructor argument for the file format
* Here, we have also added the following additional constructor arguments
  * `atom_style` - Specifies the atoms ID , type and scaled x, y, z coordinates (xs, ys, zs)
  * `guess_bonds` - Instructs MDAnalysis to guess the bonds between the atoms based on their distances. This is used when bond information is not explicitly provided in the topology file, as it is here since we are using the AIREBO force field
  * `vdwradii = {'1':1.7}` - A dictionary that maps atom types to their VDW radaii. In this case, atom type 1 is assigned a VDW radius of 1.7

In [2]:
md_universe: md_analysis.Universe = md_analysis.Universe(
    "../data/raw/cnt_atomic.data", 
    "../data/raw/cnt_breakable_bonds_dump.lammpstrj", 
    topology_format="data",
    format="lammpsdump",
    atom_style='id type x y z',
    guess_bonds=True,
    vdwradii={'1': 1.7}
)

# MD Analysis Universe.select_atoms and AtomGroup Class
* The method `Universe.select_atoms` returns an AtomGroup object
* The AtomGroup class is essentially an ordered array of atoms, with their order reflecting their order defined in the molecular topology file
* It therefore supports looping, indexing & slicing like a standard Python list
* Additionally, the class has properties representing coordinates, velocities, masses, charges, atom names, residues etc
* It also has methods for calculating properties such as distances, angles, dihedrals, center of mass, as well as methods for atomic transformation & manipulation such as translation, rotation etc

In [3]:
# Instantiate carbon atoms (atom type 1) AtomGroup object, and perform basic MD analysis
cnt: md_analysis.AtomGroup = md_universe.select_atoms("type 1")

# Basic MDAnalysis - Number of atoms in CNT and number of timestep frames 
print("Number of atoms =", cnt.n_atoms)
print("Number of frames =", md_universe.trajectory.n_frames)

# Basic MDAnalysis - Access indexes of atoms that are considered bonded by the MDAnalysis bond guesser
print("Bonded Atoms =", cnt.atoms.bonds.indices)

# Basic MDAnalysis - Loop over all the timestep frames of the trajectory
for timestep in md_universe.trajectory:
    print("Timestep frame ", timestep.frame)

# Basic MDAnalysis - Extract Atomic Positions
md_universe.atoms.positions

Number of atoms = 700
Number of frames = 286
Bonded Atoms = [[  0   2]
 [  0  23]
 [  0  56]
 ...
 [696 697]
 [696 699]
 [698 699]]
Timestep frame  0
Timestep frame  1
Timestep frame  2
Timestep frame  3
Timestep frame  4
Timestep frame  5
Timestep frame  6
Timestep frame  7
Timestep frame  8
Timestep frame  9
Timestep frame  10
Timestep frame  11
Timestep frame  12
Timestep frame  13
Timestep frame  14
Timestep frame  15
Timestep frame  16
Timestep frame  17
Timestep frame  18
Timestep frame  19
Timestep frame  20
Timestep frame  21
Timestep frame  22
Timestep frame  23
Timestep frame  24
Timestep frame  25
Timestep frame  26
Timestep frame  27
Timestep frame  28
Timestep frame  29
Timestep frame  30
Timestep frame  31
Timestep frame  32
Timestep frame  33
Timestep frame  34
Timestep frame  35
Timestep frame  36
Timestep frame  37
Timestep frame  38
Timestep frame  39
Timestep frame  40
Timestep frame  41
Timestep frame  42
Timestep frame  43
Timestep frame  44
Timestep frame  45
Time

array([[ 75.14728 ,  78.17872 ,  95.61408 ],
       [ 75.33008 ,  77.751114,  93.20232 ],
       [ 75.550476,  77.34152 ,  94.54224 ],
       ...,
       [ 84.66992 ,  82.24888 , 143.84988 ],
       [ 84.66992 ,  82.24888 , 147.60156 ],
       [ 84.85272 ,  81.82128 , 146.26175 ]], dtype=float32)

# Counting CNT Bonds
In order to measure the evolution of the number of bonds over time, we will loop over the simulation trajectory and manually extract the inter-atomic distance over time. 

We will have a nested loop within this simulation trajectory loop, looping over the indices of the atoms that are detected as bonded and calculate the distance between the two atoms.

In [ ]:
for timestep in md_universe.trajectory:
    # `cnt.atoms.bonds.indices` is a list of tuples and the expression `index_1, index_2` unpacks each tuple
    # index_1 and index_2 represent the indices of the two atoms in each bond
    for index_1, index_2 in cnt.atoms.bonds.indices:
        
        ## `md_universe.atoms.positions` provides the positions of all atoms in the current timestep as a NumPy array
        ## `md_universe.atoms.indices == index_1` creates a boolean array where the elements are True for the atom with index `index_1` and False otherwise
        ## `md_universe.atoms.positions[md_universe.atoms.indices == index_1]` uses this boolean array to filter the positions array, returning the position of the atom with index index_1.
        position_1 = md_universe.atoms.positions[md_universe.atoms.indices == index_1][0]
        position_2 = md_universe.atoms.positions[md_universe.atoms.indices == index_2][0]

        ## Calculate bond length as Euclidean distance
        bond_length: float = numpy.sqrt(numpy.sum((position_1 - position_2)**2))

        # If bond length is larger than 1.8 Å, the bond is broken


# Extract Mean Bond Lengths & Number of Bonds
Using the same method above, we will now store the mean length of the bonds and the total number of bonds in two separate lists.

In [ ]:
bond_length_vs_timestep_frame: list = []
bond_number_vs_timestep_frame: list = []

for timestep in md_universe.trajectory:
    frame: int = timestep.frame
    current_timestep_bond_lengths: list = []
    for index_1, index_2 in cnt.atoms.bonds.indices:
        position_1: ndarray = md_universe.atoms.positions[md_universe.atoms.indices == index_1]
        position_2: ndarray = md_universe.atoms.positions[md_universe.atoms.indices == index_2]

        bond_length: float = numpy.sqrt(numpy.sum((position_1 - position_2) ** 2))

        if bond_length < 1.8:
            current_timestep_bond_lengths.append(bond_length)

    mean_bond_length: float = numpy.mean(current_timestep_bond_lengths)
    number_of_bonds: int = round(len(current_timestep_bond_lengths) / 2)  # Divide by two to avoid counting twice

    bond_length_vs_timestep_frame.append([frame, mean_bond_length])
    bond_number_vs_timestep_frame.append([frame, number_of_bonds])

# Save 'bond length vs timestep frame' and 'bond number vs timestep frame' data to file
numpy.savetxt("../data/processed/bond_length_vs_timestep_frame.dat", bond_length_vs_timestep_frame)
numpy.savetxt("../data/processed/bond_number_vs_timestep_frame.dat", bond_number_vs_timestep_frame)

# Create Matplotlib graph to plot 'bond length vs timestep frame' and 'bond number vs timestep frame' data
* We will now load the data from the to .dat files into two Numpy arrays
* We will then use slicing syntax (`bond_length_vs_timestep_frame[:, 0]`) to extract all the elements in the `bond_length_vs_timestep_frame[0]` sub-array, and `bond_length_vs_timestep_frame[:, 1]` to extract all the elements in the `bond_length_vs_timestep_frame[1]` sub-array
* The same will be done for `bond_number_vs_timestep_frame`

In [ ]:
# Load data
bond_length_vs_timestep_frame: ndarray = numpy.loadtxt("../data/processed/bond_length_vs_timestep_frame.dat")
bond_number_vs_timestep_frame: ndarray = numpy.loadtxt("../data/processed/bond_number_vs_timestep_frame.dat")

# Create two line graphs/subplots
line_graphs, (bond_length_vs_timestep_frame_axes, bond_number_vs_timestep_frame_axes) = pyplot.subplots(
    2,
    1,
    figsize=(10, 6)
)
# Set background colour to black and grid coloir to white for both subplots
line_graphs.patch.set_facecolor('black')
bond_length_vs_timestep_frame_axes.set_facecolor('black')
bond_number_vs_timestep_frame_axes.set_facecolor('black')

# Set colour of ticks and spines on botha axes to white
for axes in [bond_length_vs_timestep_frame_axes, bond_number_vs_timestep_frame_axes]:
    axes.tick_params(colors='white', which='both')
    for spine in axes.spines.values():
        spine.set_edgecolor('white')


# Extract bond length vs timestep frame data and plot line graph subplot
bond_length_vs_timestep_frame_axes.plot(bond_length_vs_timestep_frame[:, 0], bond_length_vs_timestep_frame[:, 1],
                                        color='cyan')
bond_length_vs_timestep_frame_axes.set_title('CNT Bond Length vs Timestep Frame (a)', color='white')
bond_length_vs_timestep_frame_axes.set_xlabel('t (ps)', color='white')
bond_length_vs_timestep_frame_axes.set_ylabel('Bond Length (Å)', color='white')
bond_length_vs_timestep_frame_axes.set_ylim([1.35, 1.65])

# Extract number of bonds vs timestep frame data and plot line graph subplot
bond_number_vs_timestep_frame_axes.plot(bond_number_vs_timestep_frame[:, 0], bond_number_vs_timestep_frame[:, 1],
                                        color='cyan')
bond_number_vs_timestep_frame_axes.set_title('CNT Number of Bonds vs Timestep Frame (b)', color='white')
bond_number_vs_timestep_frame_axes.set_xlabel('t (ps)', color='white')
bond_number_vs_timestep_frame_axes.set_ylabel('Number of Bonds', color='white')
bond_number_vs_timestep_frame_axes.set_ylim([500, 520])

# Set x-axis limits for both plots (same for both)
bond_length_vs_timestep_frame_axes.set_xlim([0, 300])
bond_number_vs_timestep_frame_axes.set_xlim([0, 300])

# Adjust the spacing between subplots
pyplot.subplots_adjust(hspace=0.5)

line_graphs.suptitle('CNT Average Bond Length & Bond Number vs Timestep Frame', color='white')
line_graphs.text(
    0.5,
    0.0005,
    r'$\bf{Fig\ 1}$ Evolution of carbon nanotube (CNT) average bond length (a) '
    r'and bond number (b) as a function of time.',
    ha='center',
    va='center',
    color='white',
    fontsize=12
)

# Adjust layout and show line graphs
pyplot.tight_layout()
pyplot.savefig('../reports/figures/cnt_average_bond_length_bond_number_vs_timestep.png', bbox_inches='tight')
pyplot.show()

# Extract bond length distributions data at beginning of simulation, and at maximum CNT deformation

Using a similar script as before, we will extract the bond length distribution at the beginning of the simulation (let us say the 20 first frame), as well as near the maximum deformation of the CNT

In [ ]:
bond_length_distributions = []
for timstep in md_universe.trajectory:
    frame: int = timestep.frame
    current_timestep_bond_lengths: list = []
    for index_1, index_2 in cnt.atoms.bonds.indices:
        position_1: ndarray = md_universe.atoms.positions[md_universe.atoms.indices == index_1]
        position_2: ndarray = md_universe.atoms.positions[md_universe.atoms.indices == index_2]
        
        bond_length: float = numpy.sqrt(numpy.sum((position_1 - position_2) ** 2))
        
        if bond_length < 1.8:
            current_timestep_bond_lengths.append(bond_length)
            
    if frame > 0: # Ignore the first frame
        # Histogram calculation of all bond lengths and 50 bins ranging from 1.3 to 1.65 Å
        # Variable `bond_length_histo` contains counts of bond lengths in each bin
        bond_length_histo, bin_edges = numpy.histogram(current_timestep_bond_lengths, bins=50, 
                                                       range=(1.3, 1.65))
        
        # Convert bin edges to bin centers by averaging each pair of adjacent bin edges
        bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2
        
        # Normalize histogram counts by calculating bin widths & dividing histogram counts by total number of
        # bond lengths * the bin width
        # Histogram counts are normalized to obtain the probability density of each bond length.
        bin_width = bin_edges[1] = bin_edges[0]
        bond_length_histo = bond_length_histo / (numpy.sum(bond_length_histo) * bin_width)
        
        # Store bin center and normalized histogram counts by stacking vertically using `numpy.vstack`, and
        # appending to `bond_length_distributions` list
        bond_length_distributions.append(numpy.vstack([bin_centers, bond_length_histo]))
        
# Slice `bond_length_distributions` list to extract first bond length distributions for first 20 frames
# Calculate mean of these 20 distributions along the vertical axis
starting_bond_length_distribution = numpy.mean(bond_length_distributions[0:20], axis=0)

# Slice `bond_length_distributions` list to extract first bond length distributions between frames 200 & 220
maximum_deformation_bond_length_distribution = numpy.mean(bond_length_distributions[200:220], axis=0)

numpy.savetxt('../data/processed/starting_bond_length_distribution.dat', 
              starting_bond_length_distribution.T)
numpy.savetxt('../data/processed/maximum_deformation_bond_length_distribution.dat', 
              maximum_deformation_bond_length_distribution.T)

# Create Matplotlib graph to plot bond length distribution data at beginning of simulation, and at maximum CNT deformation

In [ ]:
# Load bond length distribution data and plot on custom line graph
starting_bond_length_distributions_data = numpy.loadtxt(
    '../data/processed/starting_bond_length_distribution.dat').T
maximum_deformation_bond_length_distributions_data = (numpy.loadtxt(
    '../data/processed/maximum_deformation_bond_length_distribution.dat').T)

# Extract data for plotting
starting_bin_centers: ndarray = starting_bond_length_distribution[0]
starting_histogram: ndarray = starting_bond_length_distribution[1]

deformation_bin_centers: ndarray = maximum_deformation_bond_length_distribution[0]
deformation_histogram: ndarray = maximum_deformation_bond_length_distribution[1]

# Set up the line graph
line_graph, line_graph_axes = pyplot.subplots(figsize=(10, 6))
line_graph.patch.set_facecolor('black')
line_graph_axes.set_facecolor('black')

# Stylise ticks and spines
line_graph_axes.tick_params(colors='white', which='both')
for spine in line_graph_axes.spines.values():
    spine.set_edgecolor('white')
    
# Plot the data
line_graph_axes.plot(starting_bin_centers, starting_histogram, color='cyan', label='At Start (Frames 1 - 20)')
line_graph_axes.plot(deformation_bin_centers, deformation_histogram, color='orange',
        label='During Maximum Deformation (Frames 200 - 220)')

# Set axes labels, limits and graph title
line_graph_axes.set_xlabel('Bond Length (Å)', color='white')
line_graph_axes.set_ylabel('Probability', color='white')
line_graph_axes.set_ylim([0.00, 0.13])
line_graph_axes.set_xlim([1.30, 1.65])
line_graph_axes.set_title('Carbon Nanotube Bond Length Probability Densities', color='white')

# Add legend
legend: Legend = line_graph_axes.legend(loc='upper right', frameon=False, fontsize=12)
for text in legend.get_texts():
    text.set_color('white')

# Add figure title
line_graph.text(0.5, 0.0005,
                r'$\bf{Fig\ 2}$ Carbon nanotube (CNT) bond length distribution at start of '
                r'simulation & at maximum deformation.',
                ha='center', va='center', color='white', fontsize=12)

# Show plot
pyplot.tight_layout()
pyplot.savefig('../reports/figures/cnt_bond_length_distributions.png', bbox_inches='tight')
pyplot.show()